# 1. 텍스트 감정 분석의 유용성

 - https://dbr.donga.com/article/view/1202/article_no/8891/ac/magazine
 
  - 텍스트 데이터에서만 얻을 수 있는 유용한 정보 : 사람들의 감성!
  - SNS에서 광범위한 텍스트 데이터를 얻을 수 있는데, 사람들이 자율성을 갖고 직접 올리는 글이기 때문에 감정이 쉽게 담겨있다. ==> 실시간 관심?을 쉽게 알 수 있다. (기사 참조)
  
  ### 감성어 사전 VS 머신러닝
  
  - 감성어 사전의 단점
  1. 분석 대상(도메인 : 화장품, 영화, 스포츠등)에 따라 감성 점수가 달라진다
  2. 감성의 분석대상( 화장품의 발림성 or 색감 or 가루날림 등..)의 원인을 파악하기 어렵다.
  
  ### 감성 분석을 통해 의사결정에 도움을 줄 수 있다.
  
  ### 임베딩 기법을 활용하면 비용을 절감하고 정확도를 향상시킬 수 있음!(Transfer Learning하고 비슷함)
 

# 2. 그렇다면 텍스트 데이터를 어떻게 숫자 행렬로 나타내는가 ?

In [1]:
# 처리해야 할 문장을 파이썬 리스트에 옮겨담았습니다.
sentences=['i feel hungry', 'i eat lunch', 'now i feel happy']

# 파이썬 split() 메소드를 이용해 단어 단위로 문장을 쪼개 봅니다.
word_list = 'i feel hungry'.split()
print(word_list)

['i', 'feel', 'hungry']


In [2]:
index_to_word={}  # 빈 딕셔너리를 만들어서

# 단어들을 하나씩 채워 봅니다. 채우는 순서는 일단 임의로 하였습니다. 그러나 사실 순서는 중요하지 않습니다. 
# <BOS>, <PAD>, <UNK>는 관례적으로 딕셔너리 맨 앞에 넣어줍니다. 
index_to_word[0]='<PAD>'  # 패딩용 단어
index_to_word[1]='<BOS>'  # 문장의 시작지점
index_to_word[2]='<UNK>'  # 사전에 없는(Unknown) 단어
index_to_word[3]='i'
index_to_word[4]='feel'
index_to_word[5]='hungry'
index_to_word[6]='eat'
index_to_word[7]='lunch'
index_to_word[8]='now'
index_to_word[9]='happy'

print(index_to_word)

{0: '<PAD>', 1: '<BOS>', 2: '<UNK>', 3: 'i', 4: 'feel', 5: 'hungry', 6: 'eat', 7: 'lunch', 8: 'now', 9: 'happy'}


In [3]:
word_to_index={word:index for index, word in index_to_word.items()}
print(word_to_index)

{'<PAD>': 0, '<BOS>': 1, '<UNK>': 2, 'i': 3, 'feel': 4, 'hungry': 5, 'eat': 6, 'lunch': 7, 'now': 8, 'happy': 9}


In [4]:
print(word_to_index['feel'])  # 단어 'feel'은 숫자 인덱스 4로 바뀝니다.

4


### i eat lunch 를 단어 인덱스로 변경해주기 (Encoding)

In [5]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트로 변환해 주는 함수를 만들어 봅시다.
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

print(get_encoded_sentence('i eat lunch', word_to_index))

[1, 3, 6, 7]


In [6]:
# 여러 개의 문장 리스트를 한꺼번에 숫자 텐서로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# sentences=['i feel hungry', 'i eat lunch', 'now i feel happy'] 가 아래와 같이 변환됩니다. 
encoded_sentences = get_encoded_sentences(sentences, word_to_index)
print(encoded_sentences)

[[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]]


### 반대로, Encode된 벡터를 Decode하여 다시 원래 텍스트로 복구하기

In [7]:
# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

print(get_decoded_sentence([1, 3, 4, 5], index_to_word))

i feel hungry


In [8]:
# 여러개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

# encoded_sentences=[[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]] 가 아래와 같이 변환됩니다.
print(get_decoded_sentences(encoded_sentences, index_to_word))

['i feel hungry', 'i eat lunch', 'now i feel happy']


## 하지만 위의 Encode숫자는 단순히 순서대로 넣은거라 벡터로 활용하기 어렵다...
## 그래서 사용하는 것 : Embeding Vector

# 3. Embeding Vector!

 - 단어의 의미를 나타내는 벡터를 훈련 가능한 마라미터로 변경해 주는 것.
 - Tensorflow, Pytorch에 이러한 레이어를 제공한다.
 
 ![img1](Embedingim1.png)

 ### Embeding Layer의  Input은 문장 길이가 일정해야 한다.
 - 그 이유 : RNN에서는 입력 데이터와 출력 데이터의 사이즈가 정해져 있어서, 사이즈가 다르면 학습이 불가능
 ### 따라서 Padding을 해줘야 한다.
 

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

vocab_size = len(word_to_index)  # 위 예시에서 딕셔너리에 포함된 단어 개수는 10
word_vector_dim = 4    # 위 그림과 같이 4차원의 워드벡터를 가정합니다. 

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=word_vector_dim, mask_zero=True)

# 숫자로 변환된 텍스트 데이터 [[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]] 에 Embedding 레이어를 적용합니다. 
# list 형태의 sentences는 numpy array로 변환되어야 딥러닝 레이어의 입력이 될 수 있습니다.
raw_inputs = np.array(get_encoded_sentences(sentences, word_to_index))

raw_inputs = keras.preprocessing.sequence.pad_sequences(raw_inputs,
                                                       value=word_to_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=5)
print(raw_inputs)

[[1 3 4 5 0]
 [1 3 6 7 0]
 [1 8 3 4 9]]


In [12]:
import numpy as np
import tensorflow as tf

vocab_size = len(word_to_index)  # 위 예시에서 딕셔너리에 포함된 단어 개수는 10
word_vector_dim = 4    # 그림과 같이 4차원의 워드벡터를 가정합니다.

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=word_vector_dim, mask_zero=True)

# keras.preprocessing.sequence.pad_sequences를 통해 word vector를 모두 일정길이로 맞춰주어야 
# embedding 레이어의 input이 될 수 있음에 주의해 주세요. 
raw_inputs = np.array(get_encoded_sentences(sentences, word_to_index))
raw_inputs = keras.preprocessing.sequence.pad_sequences(raw_inputs,
                                                       value=word_to_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=5)
output = embedding(raw_inputs)
print(output)

tf.Tensor(
[[[-0.00805211  0.01827027 -0.00177332  0.04897993]
  [ 0.0129673   0.04378995  0.03039024  0.03257619]
  [-0.03581922 -0.03127065 -0.01726662  0.02485519]
  [-0.03729752 -0.00223384 -0.00874346  0.01511085]
  [ 0.01752896 -0.04927242 -0.03169715 -0.02455367]]

 [[-0.00805211  0.01827027 -0.00177332  0.04897993]
  [ 0.0129673   0.04378995  0.03039024  0.03257619]
  [ 0.04579378 -0.01128112  0.02204782  0.02760938]
  [ 0.02012526 -0.00340539  0.02336298  0.0037517 ]
  [ 0.01752896 -0.04927242 -0.03169715 -0.02455367]]

 [[-0.00805211  0.01827027 -0.00177332  0.04897993]
  [-0.04342443 -0.01088833 -0.0300823  -0.0125744 ]
  [ 0.0129673   0.04378995  0.03039024  0.03257619]
  [-0.03581922 -0.03127065 -0.01726662  0.02485519]
  [-0.04062837 -0.01760133  0.01914323  0.04014159]]], shape=(3, 5, 4), dtype=float32)


### shaped의 (3,5,4) 의미
3 입력 문장 개수
5 입력 문장 최대 길이
4 워드 벡터 차원 수

# 4. Sequence 를 다루는 RNN

 #### 텍스트 데이터도 시퀀스 데이터다.
 
 - at time=0s : 듣는이의 귀에 들어온 input='i'
 - at time=1s : 듣는이의 귀에 들어온 input='feel'
 - at time=2s : 듣는이의 귀에 들어온 input='hungry'
 
 ![im2](stateim1.png)

손님 : 이전 시점에 어떤 선택을 했는지 기억하고 말한다.

점원 : (Stateless)에서 이전에 대화를 기억 못해서 손님이 처음부터다시 다 얘기해줘야한다.

RNN 참고  : https://youtu.be/-SHPG_KMUkQ?t=2